In [1]:
import requests
import collections
from io import StringIO

#imprt the necessary calculations for everything we will use
from datetime import datetime, timedelta
import time
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from netCDF4 import Dataset, num2date, date2num
import numpy as np
from scipy.ndimage import gaussian_filter
from shapely.geometry.polygon import LinearRing
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import xlrd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.stats import linregress

################################################################################
from IPython import embed
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from shapely.geometry import Point
import cartopy
from shapely.prepared import prep
import seaborn as sns
import collections

In [13]:
#blue marble pathway
os.environ["CARTOPY_USER_BACKGROUNDS"] = '/Users/cjmasiel/Desktop/Cartopy/BG/'

In [31]:
#this is the function that creates the map background of the North Atlantic Domain
def create_map_background():
    dataproj = ccrs.PlateCarree()
    fig=plt.figure(figsize=(25, 25))
    ax=plt.subplot(111, projection=dataproj)
    ax.set_extent([-10, -105, 5, 50], ccrs.PlateCarree())
    ax.coastlines('50m', linewidth=1.5)
    ax.add_feature(cfeature.STATES, linewidth=1.0)
    ax.add_feature(cfeature.BORDERS, linewidth=1.0)
    
    gl = ax.gridlines(color='gray', alpha=0.5, draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'fontsize': 16}
    gl.ylabel_style = {'fontsize': 16}
    gl.xlocator = mticker.FixedLocator([0, -10, -20, -30, -40, -50, -60, -70, -80, -90, -100, -110])
    gl.ylocator = mticker.FixedLocator([0, 10, 20, 30, 40, 50])
    gl.xformatter = LongitudeFormatter(zero_direction_label=True)
    gl.yformatter = LatitudeFormatter()
    
    ax.xaxis.set_major_formatter(gl.xformatter)
    ax.yaxis.set_major_formatter(gl.yformatter)
    ###############################################################################################
    ax.background_img(name='BM', resolution='high')
    ###############################################################################################
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color = '#5ebaff', label = 'Tropical Depression')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#00faf4', label = 'Tropical Storm')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#ffffcc', label = 'Category 1')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#ffe775', label = 'Category 2')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#ffc140', label = 'Category 3')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#ff8f20', label = 'Category 4')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#ff6060', label = 'Category 5')
    plt.scatter(-180,-40,alpha = 1.0, s = 50,color='#5ebaff', marker  = 'v', label = 'Extra-Tropical')
    plt.scatter(-180,-40,alpha = 1.0, s = 50, color='#00faf4', marker = 'v', label = 'Sub-Tropical Storm')
    plt.legend(loc = 'upper right', fontsize = '20')
    return fig, ax

In [ ]:
# #Fetch data from the URL

# storms = ['01','02','03']

# for i in range(len(storms)):
#     url = f'https://ftp.nhc.noaa.gov/atcf/btk/bal{storms[i]}2024.dat'
#     response = requests.get(url)
#     data = response.text
    
#     # Process the data as if it's a file
#     ds = StringIO(data)
#     storm_data_ATCF_BT = collections.defaultdict(list)
    
#     # Set your year bounds and other parameters
#     lower_bound = 2020
#     upper_bound = 2024
    
#     for line in ds:
#         if not line.startswith("AL"):  # Skip header lines
#             continue
        
#         values = line.split(",")
        
#         code = values[1].strip()
#         datetime_str = values[2].strip()
#         year = int(datetime_str[:4])
        
#         latitude = float(values[6][:-1]) / 10.0  # Adjust latitude format
#         longitude = -float(values[7][:-1]) / 10.0  # Adjust longitude format
#         vmax = float(values[8])
#         nature = values[10].strip()
#         pressure = values[9].strip()
#         month = int(datetime_str[4:6])
#         date_str = datetime_str.strip()
    
#         if (nature in ('TD', 'TS', 'HU')):
#             storm_data_ATCF_BT[code].append({
#                 "latitude": latitude,
#                 "longitude": longitude,
#                 "vmax": vmax,
#                 "datetime": datetime_str,
#                 "pressure": pressure,
#                 "nature": nature,
#                 "month": month,
#                 "date_str": date_str
#             })

# # Print the first elements of storm data for a valid key
# if storm_data_ATCF_BT:
#     first_key = next(iter(storm_data_ATCF_BT))
#     print("First elements of storm data for", first_key, storm_data_ATCF_BT[first_key])
# else:
    # print("No data found matching the criteria.")

In [7]:
# List of storm numbers to fetch
storms = ['01', '02', '03']

# Dictionary to store storm data
storm_data_ATCF_BT = collections.defaultdict(list)

# Fetch and process data for each storm
for storm in storms:
    url = f'https://ftp.nhc.noaa.gov/atcf/btk/bal{storm}2024.dat'
    response = requests.get(url)
    data = response.text
    
    # Process the data as if it's a file
    ds = StringIO(data)
    
    for line in ds:
        if not line.startswith("AL"):  # Skip header lines
            continue
        
        values = line.split(",")
        
        code = values[1].strip()
        datetime_str = values[2].strip()
        year = int(datetime_str[:4])
        
        latitude = float(values[6][:-1]) / 10.0  # Adjust latitude format
        longitude = -float(values[7][:-1]) / 10.0  # Adjust longitude format
        vmax = float(values[8])
        nature = values[10].strip()
        pressure = values[9].strip()
        month = int(datetime_str[4:6])
        date_str = datetime_str.strip()
    
        storm_data_ATCF_BT[code].append({
                "latitude": latitude,
                "longitude": longitude,
                "vmax": vmax,
                "datetime": datetime_str,
                "pressure": pressure,
                "nature": nature,
                "month": month,
                "date_str": date_str
        })

#Print the first elements of storm data for a valid key
if storm_data_ATCF_BT:
    for key in storm_data_ATCF_BT:
        print(f"First elements of storm data for {key}: {storm_data_ATCF_BT[key][:5]}")
else:
    print("No data found matching the criteria.")

First elements of storm data for 01: [{'latitude': 17.8, 'longitude': -89.1, 'vmax': 25.0, 'datetime': '2024061612', 'pressure': '1006', 'nature': 'DB', 'month': 6, 'date_str': '2024061612'}, {'latitude': 18.2, 'longitude': -89.9, 'vmax': 25.0, 'datetime': '2024061618', 'pressure': '1006', 'nature': 'DB', 'month': 6, 'date_str': '2024061618'}, {'latitude': 18.6, 'longitude': -90.7, 'vmax': 30.0, 'datetime': '2024061700', 'pressure': '1005', 'nature': 'DB', 'month': 6, 'date_str': '2024061700'}, {'latitude': 19.0, 'longitude': -91.6, 'vmax': 30.0, 'datetime': '2024061706', 'pressure': '1005', 'nature': 'DB', 'month': 6, 'date_str': '2024061706'}, {'latitude': 19.4, 'longitude': -92.5, 'vmax': 30.0, 'datetime': '2024061712', 'pressure': '1004', 'nature': 'DB', 'month': 6, 'date_str': '2024061712'}]
First elements of storm data for 02: [{'latitude': 9.4, 'longitude': -25.5, 'vmax': 15.0, 'datetime': '2024062600', 'pressure': '0', 'nature': 'DB', 'month': 6, 'date_str': '2024062600'}, {'la

In [ ]:
storm_data_ATCF_BT

In [43]:
def plotstorm(x):
    Lat = []
    Long = []
    LatStorm = []
    LongStorm = []
    LatStorm1 = []
    LongStorm1 = []
    Lat1 = []
    Long1 = []
    Lat2 = []
    Long2 = []
    Lat3 = []
    Long3 = []
    Lat4 = []
    Long4 = []
    Lat5 = []
    Long5 = []
    LatSS = []
    LongSS = []
    LatEX = []
    LongEX = []

    #Process storm data
    for code, data in storm_data_ATCF_BT.items():
        if data:  # Ensure there is data for the storm
            latitude  = [entry["latitude"] for entry in data]
            longitude = [entry["longitude"] for entry in data]
            vmax      = [entry["vmax"] for entry in data]
            nature    = [entry["nature"] for entry in data]
            time      = [entry["date_str"] for entry in data]
    ###################################################################################################
            if code == x:
                for i in range(len(vmax)):
                    LatStorm1.append(latitude[i])
                    LongStorm1.append(longitude[i])
                    if vmax[i] < 34 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat.append(latitude[i])
                        Long.append(longitude[i])
                    if 34 <= vmax[i] < 63 and nature[i] in ('TD', 'TS', 'HU'):
                        LatStorm.append(latitude[i])
                        LongStorm.append(longitude[i])
                    if 63 < vmax[i] <= 83 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat1.append(latitude[i])
                        Long1.append(longitude[i])
                    if 83 < vmax[i] <= 95 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat2.append(latitude[i])
                        Long2.append(longitude[i])
                    if 95 < vmax[i] <= 112 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat3.append(latitude[i])
                        Long3.append(longitude[i])
                    if 112 < vmax[i] <= 136 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat4.append(latitude[i])
                        Long4.append(longitude[i])
                    if vmax[i] > 136 and nature[i] in ('TD', 'TS', 'HU'):
                        Lat5.append(latitude[i])
                        Long5.append(longitude[i])
                    if nature[i] in ('SS'):
                        LatSS.append(latitude[i])
                        LongSS.append(longitude[i])
                    if nature[i] in ('EX','LO','WV','SD','DB'):
                        LatEX.append(latitude[i])
                        LongEX.append(longitude[i])  

    
    #Below is the plotting code \n",
    plt.plot(LongStorm1, LatStorm1, alpha = 1.0, color='grey', linewidth=1,zorder=1)
    plt.scatter(Long, Lat, color = '#5ebaff',s=100, label = 'Tropical Depression',zorder=2)
    plt.scatter(LongStorm,LatStorm, color='#00faf4',s=100, label = 'Tropical Storm',zorder=3)
    plt.scatter(Long1,Lat1,alpha = 1.0,s=100, color='#ffffcc', label = 'Category 1',zorder=4)
    plt.scatter(Long2,Lat2,alpha = 1.0, s=100,color='#ffe775', label = 'Category 2',zorder=5)
    plt.scatter(Long3,Lat3,alpha = 1.0,s=100, color='#ffc140', label = 'Category 3',zorder=6)
    plt.scatter(Long4,Lat4,alpha = 1.0,s=100, color='#ff8f20', label = 'Category 4',zorder=7)
    plt.scatter(Long5,Lat5,alpha = 1.0,s=100, color='#ff6060', label = 'Category 5',zorder=8)
    plt.scatter(LongEX,LatEX,alpha = 1.0,s=25, color='#5ebaff', marker  = 'v', label = 'Extra-Tropical',zorder=9)
    plt.scatter(LongSS,LatSS,alpha = 1.0,s=25, color='#00faf4', marker = 'v', label = 'Sub-Tropical Storm',zorder=10)
    
    plt.title(f'2024 Atlantic Hurricane Season', loc = 'left',fontsize = 24)
    plt.title('Product by: Cameron Masiello', loc = 'right', fontsize = 24)

In [ ]:
fig, ax = create_map_background()

for storm in storms:
    plotstorm(storm)